<a href="https://colab.research.google.com/github/joshcova/NLP_Workshop/blob/main/03_NLP_Spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text representation in Spacy

`spacy` is a popular and highly efficient open-source library for advanced Natural Language Processing (NLP) in Python. It's designed for production use and offers capabilities for a wide range of NLP tasks, including tokenization, named entity recognition (NER), part-of-speech tagging (POS) etc.

When `spacy` processes text, it converts the text into a Doc object. This Doc object is essentially a sequence of Token objects, which are the fundamental building blocks for all subsequent NLP operations. Each Token represents an individual word, punctuation mark, or symbol.

In NLP, a corpus refers to a large and structured set of texts used for linguistic analysis. A document is an individual text within that corpus. In spaCy, the Doc object represents an individual document that has been processed by the nlp pipeline.

Tokens are the smallest units of text obtained after splitting a sentence or phrase. They are typically individual words, punctuation marks, or symbols, separated by whitespace or specific rules defined by the tokenizer.

In [3]:
import spacy

In [5]:
# now we need to do some downloading, we download Spacy's language and trained pipelines, there are different sizes for English: small (en_core_web_sm), medium (en_core_web_md), large (en_core_web_lg)
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 12.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
# load the Spacy pipeline into your environment
nlp = spacy.load("en_core_web_md")

In [7]:
doc = nlp("This is a sentence. $20 is the price.")

In [ ]:
# You can see that some tokens are words, some are not

for token in doc:
  print(token)

In [ ]:
# You can check the Part-of-Speech (POS) and the Lemma

doc = nlp("This is a better sentence.")
for token in doc:
    print(token, " | ", token.pos_, " | ", token.lemma_)

In [ ]:
# We can also do some Named Entity Recognition (NER)

doc = nlp("Elon Musk bought Twitter for $44 billion.")

for ent in doc.ents:
    print(ent.text, " | ", ent.label_, " | ", spacy.explain(ent.label_))

In [61]:
import pandas as pd

# export to csv

data = []
for ent in doc.ents:
    data.append({
        "text": ent.text,
        "label": ent.label_,
        "explanation": spacy.explain(ent.label_)
    })

df = pd.DataFrame(data)

## Stopwords

The Spacy library offers different stop words list for different languages

In [70]:
from spacy.lang.en.stop_words import STOP_WORDS
#from spacy.lang.de.stop_words import STOP_WORDS

In [ ]:
doc = nlp("This is a sample sentence, showing off the stop words filtration.")

for token in doc:
    if token.is_stop:
        print(token.text)

Let's bring it all together using the dataframe that we used yesterday!

In [26]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/joshcova/NLP_Workshop/refs/heads/main/data/brexit_data.csv")

In [27]:
df_select = df[["text","party"]]

In [29]:
# let's sample some rows from the dataframe

df_select = df_select.sample(100, random_state=1)

In [30]:
def preprocess(text):
    doc = nlp(text)
    no_stop_words = [token.text for token in doc if not token.is_stop and not token.is_punct]
    return no_stop_words

In [31]:
df_select["text_no_stop_words"] = df_select["text"].apply(preprocess)

In [32]:
def pos_tag_text(text):
    doc = nlp(text)
    return [(token.text, token.pos_, token.tag_) for token in doc]

In [33]:
df_select["pos"] = df_select["text"].apply(pos_tag_text)

## Sentiment analysis

There are different sentiment dictionaries, which list "positive" and "negative" words (e.g. Lexicoder Sentiment Dictionary).

These are called **word-list based sentiment analysis**

Here we will showcase how such a sentiment dictionary analysis works by using the [Afinn sentiment analysis](https://github.com/fnielsen/afinn) and the [VADER](https://vadersentiment.readthedocs.io/en/latest/)  sentiment analysis tool

In [28]:
pip install Afinn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Afinn: filename=afinn-0.1-py3-none-any.whl size=53431 sha256=f79560413f68122539a3a2c5f9f9836d58d67337ec411a383cf465e42b964538
  Stored in directory: /root/.cache/pip/wheels/f9/72/27/74994e77200dae3d6aea2b546264500cee21f738c51241320b
Successfully built Afinn


In [29]:
from afinn import Afinn

In [78]:
texts = [
    "This is a good movie",
    "This is not good at all",
    "What an awful experience!"
]


In [34]:
afinn = Afinn()

In [40]:
texts_df = pd.DataFrame(texts, columns=["text"])

In [42]:
texts_df["polarity_score"] = texts_df["text"].apply(afinn.score)

Another commoly used rule-based dictionary especially for social media texts is the [VADER](https://vadersentiment.readthedocs.io/en/latest/) (Valence Aware Dictionary and sEntiment Reasoner) sentiment analysis tool

In [ ]:

pip install vaderSentiment

In [40]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [41]:
analyzer = SentimentIntensityAnalyzer()

In [83]:
for t in texts:
  score = analyzer.polarity_scores(t)
  print(score)

{'neg': 0.0, 'neu': 0.58, 'pos': 0.42, 'compound': 0.4404}
{'neg': 0.325, 'neu': 0.675, 'pos': 0.0, 'compound': -0.3412}
{'neg': 0.523, 'neu': 0.477, 'pos': 0.0, 'compound': -0.5093}


In [45]:
df_select["sentiment_scores"] = df_select["text"].apply(analyzer.polarity_scores)